In [ ]:
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
from dbtools import dbtools as db
from dbtools import load
from preprocess_tools import io, aligner, reslicer

# Database conection

In [ ]:
try:
    conn = db.connect()
    print("Connected to the database")
except Exception as error:
    print(error)

# Measurements file paths

List of paths to the XCT measurements you aim to align

In [ ]:
paths = [Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\14\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\15\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\16\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\17\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\21\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\22\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\23\volume_eq'),
         Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\probetas\1\24\volume_eq')]

#check that the paths exist
for path in paths:
    if not path.exists():
        print(f"Path {path} does not exist")

# Measurement type id

Get the measurement id of each measurement

In [ ]:
measurements_table = db.get_data('measurements')

measurmenttype_ids = []

for path in paths:
    row = measurements_table[measurements_table['file_path_measurement'] == str(path)]
    if not row.empty:
        measurmenttype_ids.append(int(row['measurementtype_id_measurement'].values[0]))
    else:
        print(f"Measurement not found for path: {path}")

print("Measurement type ids:", measurmenttype_ids)

# Sample names

Get the samples names in the measurements

In [ ]:
measurements_samples_table = db.relation_metadata('samples','measurements','sample_measurements')

measurements_samples_table

In [ ]:

sample_names = []

for path in paths:
    sub_table = measurements_samples_table[measurements_samples_table['file_path_measurement'] == str(path)]
    if not sub_table.empty:
        sample_names.append(sub_table['name_sample'].values)
    else:
        print(f"Measurement not found for path: {path}")

print("Measurement sample names:", sample_names)

# Alignment and saving

Loop that loads each measurement, aligns the volume and loads it in the database.

In [ ]:
for i, path in enumerate(paths):
    print(f"Processing {path} with measurement type id {measurmenttype_ids[i]}")
    
    # Load the data
    volume = io.load_tif(path)
    
    # Align the volume
    volume = aligner.main(volume)

    #reslice to frontal 
    volume = reslicer.rotate_90(volume, clockwise=True)
    volume = reslicer.reslice(volume,'Top')
    
    # Save the aligned volume as .tif if is less than 4GB if its more as a folder
    if volume.nbytes < 4 * 1024**3:  # Check if the size is less than 4GB
        aligned_path = path.parent / f"{path.name}_aligned.tif"
        file_type = 'tif'
    else:
        # Save as a folder
        aligned_path = path.parent / f"{path.name}_aligned"
        if not aligned_path.exists():
            aligned_path.mkdir(parents=True, exist_ok=True)
        file_type = 'tif folder'
    
    print(f"Saving aligned volume to {aligned_path}")

    io.save_tif(aligned_path,volume)

    #data for the database

    height, width, depth = volume.shape

    dtype = str(volume.dtype)

    samples = list(sample_names[i])

    equalized = True

    transformation = 'Frontwall aligned with axes, rotated 90 degrees clockwise and resliced Top. Done UTVSCT-PREPROCESING toolkit 0.1.14, aligner.main function used.'

    print(f"Inserting measurement for path {aligned_path} with type {file_type}, height {height}, width {width}, depth {depth}, dtype {dtype}, samples {samples}, equalized {equalized}, transformation {transformation}")

    # Save the aligned volume in the database
    inserted_id = load.load_xct_measurement(conn,str(aligned_path),measurmenttype_ids[i],height,width,depth,dtype,file_type,samples,True, equalized,['z','y','x'],str(path),transformation)

    print(f"Inserted measurement with id {inserted_id} for path {aligned_path}")